In [170]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Desktop/Continuous MPS/MatrixProductBP.jl`


In [171]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, TensorTrains, SparseArrays
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays
using JLD2
include("bp_fourier.jl");

In [172]:
seed = 4
rng = MersenneTwister(seed)

T = 4
# N = 3
# A = [0 1 1; 
#      1 0 0;
#      1 0 0;]
# N = 4
# A = [0 1 0 0; 
#      1 0 1 0; 
#      0 1 0 1; 
#      0 0 1 0]
# N = 7
# A = [0 1 0 0 0 0 0; 
#      1 0 1 0 0 0 0; 
#      0 1 0 1 0 0 0;
#      0 0 1 0 1 0 0;
#      0 0 0 1 0 1 0;
#      0 0 0 0 1 0 1;
#      0 0 0 0 0 1 0]
# N = 4
# A = [0 1 1 1;
#      1 0 0 1;
#      1 0 0 1;
#      1 1 1 0]
# !!! Here the calculations are wrong for edge 1->2 !!!
N = 5
A = [0 1 1 1 1; 
     1 0 0 0 0; 
     1 0 0 0 0; 
     1 0 0 0 0; 
     1 0 0 0 0]
# N = 6
# A = [0 1 1 1 0 0; 
#      1 0 0 0 1 0; 
#      1 0 0 0 0 1; 
#      1 0 0 0 0 0; 
#      0 1 0 0 0 0; 
#      0 0 1 0 0 0]
# N = 7
# A = [0 1 1 1 1 0 1;
#      1 0 0 0 1 0 0; 
#      1 0 0 0 0 1 0; 
#      1 0 0 0 0 0 0; 
#      1 1 0 0 0 0 0; 
#      0 0 1 0 0 0 0; 
#      1 0 0 0 0 0 0]
# N = 8
# A = [0 1 1 1 0 0 1 0;
#      1 0 0 0 1 0 0 1; 
#      1 0 0 0 0 1 0 0; 
#      1 0 0 0 0 0 0 0; 
#      0 1 0 0 0 0 0 0; 
#      0 0 1 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0;
#      0 1 0 0 0 0 0 0]

g = IndexedBiDiGraph(A)

# N = 20
# c = 3
# gg = erdos_renyi(N, c/N; seed)
# g = IndexedBiDiGraph(gg)
# A = zeros(Int,N,N)
# for i in CartesianIndices(A)
#      if !iszero(g.A[i])
#          A[i] = 1
#      end
# end

β = 1.0
J = 1
h = 0.0
m⁰ = 1.0
K = 100

svd_trunc=TruncBond(4)

A

5×5 Matrix{Int64}:
 0  1  1  1  1
 1  0  0  0  0
 1  0  0  0  0
 1  0  0  0  0
 1  0  0  0  0

In [173]:
w = [fill(IntegerGlauberFactor([J*A[j,i] for j in inneighbors(g,i)], h, β), T+1) for i in vertices(g)]
ϕᵢ = [t == 0 ? [(1-m⁰)/2, (1+m⁰)/2] : ones(2) for t in 0:T]
# ϕᵢ = [ones(2) for t in 0:T]
bp = mpbp(g, w, fill(2,N), T; ϕ=fill(ϕᵢ,N))

for μ in bp.μ
    for a in μ
        a.=rand.(rng)
        # a.=1.0
    end
end
# bp_fourier = deepcopy(bp)
iterate!(bp, maxiter=2; svd_trunc)

msg = deepcopy(bp.μ)
# jldsave("messages.jld2"; msg)

bp_fourier = deepcopy(bp)
# iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/100; svd_trunc)
;

Running MPBP: iter 2    Time: 0:00:00

In [174]:
# for (k,ed) in enumerate(edges(bp.g))
#     display(ed)
#     display(bp.μ[idx(ed)][3])
# end

In [175]:
iterate!(bp,maxiter=1; svd_trunc)

(1, CB_BP{ProgressUnknown, MatrixProductBP.var"#153#156"}(ProgressUnknown(false, false, 0, ProgressMeter.ProgressCore(:green, "Running MPBP: iter", 0.1, true, 0, VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(25) open, 0 bytes waiting)), VSCodeServer.io_send_callback), false, 1, 1, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (136580681402832, 136578318886656, 136580509668032)), 0, 1, false, false, 1.737537473363975e9, 1.737537473363975e9, 1.737537473363975e9)), [[[2.0, 1.9996646498695334, 1.9548049198724362, 1.9544998820940054, 1.9136950302803462], [2.0, 1.880797077977883, 1.8805416772783212, 1.8463767690747763, 1.8461444540853782], [2.0, 1.880797077977883, 1.8805416772783212, 1.8463767690747763, 1.8461444540853782], [2.0, 1.8807970779778824, 1.88054167727832, 1.8463767690747765, 1.8461444

In [176]:
iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/50; svd_trunc=TruncThresh(0.0))
;

8×8×7×2 OffsetArray(::Array{ComplexF64, 4}, 1:8, 1:8, 1:7, 1:2) with eltype ComplexF64 with indices 1:8×1:8×1:7×1:2:
[:, :, 1, 1] =
    0.0264269-5.93667e-17im  …     0.0116739-3.44342e-17im
  1.36273e-16+0.0813337im        7.15823e-17+0.0359287im
  1.36272e-16+0.081332im         7.15821e-17+0.0359279im
    -0.250314+2.7648e-16im         -0.110575+1.14452e-16im
  1.36265e-16+0.081323im         7.15782e-17+0.035924im
    -0.250287+2.76475e-16im  …     -0.110563+1.14451e-16im
    -0.250285+2.76474e-16im        -0.110562+1.14451e-16im
 -4.11083e-16-0.770297im        -2.64932e-17-0.340274im

[:, :, 2, 1] =
   -0.0280918-6.31132e-17im  …    -0.0124094-3.66073e-17im
 -1.44877e-16+0.0864576im       -7.61034e-17+0.0381922im
 -1.44878e-16+0.086459im        -7.61038e-17+0.0381927im
     0.266093+2.93957e-16im         0.117545+1.21695e-16im
 -1.44886e-16+0.0864696im       -7.61084e-17+0.0381974im
     0.266126+2.93963e-16im  …       0.11756+1.21695e-16im
     0.266128+2.93963e-16im         0.1175

8×8×7×2 OffsetArray(::Array{ComplexF64, 4}, 1:8, 1:8, 1:7, 1:2) with eltype ComplexF64 with indices 1:8×1:8×1:7×1:2:
[:, :, 1, 1] =
    0.0264269-5.93667e-17im  …     0.0116739-3.44342e-17im
  1.36273e-16+0.0813337im        7.15823e-17+0.0359287im
  1.36272e-16+0.081332im         7.15821e-17+0.0359279im
    -0.250314+2.7648e-16im         -0.110575+1.14452e-16im
  1.36265e-16+0.081323im         7.15782e-17+0.035924im
    -0.250287+2.76475e-16im  …     -0.110563+1.14451e-16im
    -0.250285+2.76474e-16im        -0.110562+1.14451e-16im
 -4.11083e-16-0.770297im        -2.64932e-17-0.340274im

[:, :, 2, 1] =
   -0.0280918-6.31132e-17im  …    -0.0124094-3.66073e-17im
 -1.44877e-16+0.0864576im       -7.61034e-17+0.0381922im
 -1.44878e-16+0.086459im        -7.61038e-17+0.0381927im
     0.266093+2.93957e-16im         0.117545+1.21695e-16im
 -1.44886e-16+0.0864696im       -7.61084e-17+0.0381974im
     0.266126+2.93963e-16im  …       0.11756+1.21695e-16im
     0.266128+2.93963e-16im         0.1175

8×8×7×2 OffsetArray(::Array{ComplexF64, 4}, 1:8, 1:8, 1:7, 1:2) with eltype ComplexF64 with indices 1:8×1:8×1:7×1:2:
[:, :, 1, 1] =
    0.0264263-5.93661e-17im  …     0.0116737-3.44339e-17im
  1.36272e-16+0.0813318im        7.15822e-17+0.0359279im
  1.36272e-16+0.0813318im        7.15822e-17+0.0359279im
    -0.250314+2.7648e-16im         -0.110575+1.14452e-16im
  1.36265e-16+0.0813228im        7.15783e-17+0.0359239im
    -0.250286+2.76475e-16im  …     -0.110562+1.14452e-16im
    -0.250286+2.76475e-16im        -0.110562+1.14452e-16im
 -4.11083e-16-0.770302im        -2.64932e-17-0.340276im

[:, :, 2, 1] =
   -0.0280925-6.31141e-17im  …    -0.0124097-3.66078e-17im
 -1.44879e-16+0.0864598im       -7.61041e-17+0.0381931im
 -1.44879e-16+0.0864598im       -7.61041e-17+0.0381931im
     0.266096+2.93958e-16im         0.117546+1.21695e-16im
 -1.44887e-16+0.0864705im       -7.61088e-17+0.0381978im
     0.266129+2.93964e-16im  …      0.117561+1.21695e-16im
     0.266129+2.93964e-16im         0.117

8×8×7×2 OffsetArray(::Array{ComplexF64, 4}, 1:8, 1:8, 1:7, 1:2) with eltype ComplexF64 with indices 1:8×1:8×1:7×1:2:
[:, :, 1, 1] =
     0.026426-5.93632e-17im  …    0.0116735-3.44322e-17im
  1.36273e-16+0.0813249im       7.15829e-17+0.0359248im
  1.36273e-16+0.0813249im       7.15829e-17+0.0359248im
    -0.250303+2.76486e-16im        -0.11057+1.1446e-16im
  1.36273e-16+0.0813248im       7.15829e-17+0.0359248im
    -0.250303+2.76486e-16im  …     -0.11057+1.1446e-16im
    -0.250303+2.76486e-16im        -0.11057+1.1446e-16im
 -4.11095e-16-0.770358im        -2.6494e-17-0.340301im

[:, :, 2, 1] =
   -0.0280952-6.31232e-17im  …    -0.0124109-3.66131e-17im
 -1.44892e-16+0.0864757im       -7.61109e-17+0.0382002im
 -1.44892e-16+0.0864757im       -7.61109e-17+0.0382002im
     0.266136+2.9398e-16im          0.117564+1.21698e-16im
 -1.44892e-16+0.0864758im       -7.61109e-17+0.0382002im
     0.266136+2.9398e-16im   …      0.117564+1.21698e-16im
     0.266136+2.9398e-16im          0.117564+1.21698

1×1×7×2 OffsetArray(::Array{ComplexF64, 4}, 1:1, 1:1, 1:7, 1:2) with eltype ComplexF64 with indices 1:1×1:1×1:7×1:2:
[:, :, 1, 1] =
 0.996059940722142 + 0.0im

[:, :, 2, 1] =
 0.9982469420662922 + 0.0im

[:, :, 3, 1] =
 0.9995614471079514 + 0.0im

[:, :, 4, 1] =
 1.0 + 0.0im

[:, :, 5, 1] =
 0.9995614471079514 + 0.0im

[:, :, 6, 1] =
 0.9982469420662922 + 0.0im

[:, :, 7, 1] =
 0.996059940722142 + 0.0im

[:, :, 1, 2] =
 0.996059940722142 + 0.0im

[:, :, 2, 2] =
 0.9982469420662922 + 0.0im

[:, :, 3, 2] =
 0.9995614471079514 + 0.0im

[:, :, 4, 2] =
 1.0 + 0.0im

[:, :, 5, 2] =
 0.9995614471079514 + 0.0im

[:, :, 6, 2] =
 0.9982469420662922 + 0.0im

[:, :, 7, 2] =
 0.996059940722142 + 0.0im

1×1×7×2 OffsetArray(::Array{ComplexF64, 4}, 1:1, 1:1, 1:7, 1:2) with eltype ComplexF64 with indices 1:1×1:1×1:7×1:2:
[:, :, 1, 1] =
 0.996059940722142 + 0.0im

[:, :, 2, 1] =
 0.9982469420662922 + 0.0im

[:, :, 3, 1] =
 0.9995614471079514 + 0.0im

[:, :, 4, 1] =
 1.0 + 0.0im

[:, :, 5, 1] =
 0.9995614471079514 + 0.0im

[:, :, 6, 1] =
 0.9982469420662922 + 0.0im

[:, :, 7, 1] =
 0.996059940722142 + 0.0im

[:, :, 1, 2] =
 0.996059940722142 + 0.0im

[:, :, 2, 2] =
 0.9982469420662922 + 0.0im

[:, :, 3, 2] =
 0.9995614471079514 + 0.0im

[:, :, 4, 2] =
 1.0 + 0.0im

[:, :, 5, 2] =
 0.9995614471079514 + 0.0im

[:, :, 6, 2] =
 0.9982469420662922 + 0.0im

[:, :, 7, 2] =
 0.996059940722142 + 0.0im

1×1×7×2 OffsetArray(::Array{ComplexF64, 4}, 1:1, 1:1, 1:7, 1:2) with eltype ComplexF64 with indices 1:1×1:1×1:7×1:2:
[:, :, 1, 1] =
 0.996059940722142 + 0.0im

[:, :, 2, 1] =
 0.9982469420662922 + 0.0im

[:, :, 3, 1] =
 0.9995614471079514 + 0.0im

[:, :, 4, 1] =
 1.0 + 0.0im

[:, :, 5, 1] =
 0.9995614471079514 + 0.0im

[:, :, 6, 1] =
 0.9982469420662922 + 0.0im

[:, :, 7, 1] =
 0.996059940722142 + 0.0im

[:, :, 1, 2] =
 0.996059940722142 + 0.0im

[:, :, 2, 2] =
 0.9982469420662922 + 0.0im

[:, :, 3, 2] =
 0.9995614471079514 + 0.0im

[:, :, 4, 2] =
 1.0 + 0.0im

[:, :, 5, 2] =
 0.9995614471079514 + 0.0im

[:, :, 6, 2] =
 0.9982469420662922 + 0.0im

[:, :, 7, 2] =
 0.996059940722142 + 0.0im

1×1×7×2 OffsetArray(::Array{ComplexF64, 4}, 1:1, 1:1, 1:7, 1:2) with eltype ComplexF64 with indices 1:1×1:1×1:7×1:2:
[:, :, 1, 1] =
 0.996059940722142 + 0.0im

[:, :, 2, 1] =
 0.9982469420662922 + 0.0im

[:, :, 3, 1] =
 0.9995614471079514 + 0.0im

[:, :, 4, 1] =
 1.0 + 0.0im

[:, :, 5, 1] =
 0.9995614471079514 + 0.0im

[:, :, 6, 1] =
 0.9982469420662922 + 0.0im

[:, :, 7, 1] =
 0.996059940722142 + 0.0im

[:, :, 1, 2] =
 0.996059940722142 + 0.0im

[:, :, 2, 2] =
 0.9982469420662922 + 0.0im

[:, :, 3, 2] =
 0.9995614471079514 + 0.0im

[:, :, 4, 2] =
 1.0 + 0.0im

[:, :, 5, 2] =
 0.9995614471079514 + 0.0im

[:, :, 6, 2] =
 0.9982469420662922 + 0.0im

[:, :, 7, 2] =
 0.996059940722142 + 0.0im

In [177]:
import MatrixProductBP: marginalize
function marginalize(A::MPEM2{F}, var) where F
    if var==1
        return MPEM1{F}([@tullio b[m, n, xi] := a[m, n, xi, xj] for a in A]; z = A.z)
    elseif var==2
        return MPEM1{F}([@tullio b[m, n, xj] := a[m, n, xi, xj] for a in A]; z = A.z)
    end
end;

In [178]:
er_b = []
ptr = []
for i in vertices(bp.g)
    margs = marginals(bp.b[i])
    margs_fourier = marginals(bp_fourier.b[i])
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        ind = argmax(abs.(marg))
        push!(er_b, abs(marg_fourier[ind]-marg[ind]))
        push!(ptr, (i, t))
    end
end

display(maximum(er_b))
display(ptr[argmax(er_b)])

0.0020143341990481733

(1, 5)

In [179]:
er_μ = []
ptr = []
for e in edges(bp.g)
    μ_marg = marginalize(bp.μ[idx(e)],1)
    margs = marginals(μ_marg)
    μ_fourier_marg = marginalize(bp_fourier.μ[idx(e)],1)
    margs_fourier = marginals(μ_fourier_marg)
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        push!(er_μ, abs(marg_fourier[1]-marg[1]))
        push!(ptr, (e, t))
    end
end

display(maximum(er_μ))
display(ptr[argmax(er_μ)])

0.0019034305960192038

(Indexed Edge 1 => 5 with index 4, 4)

In [180]:
node = 2
time = 3

ed = inedges(bp.g, node) |> first
@show ed
μ = bp.μ[idx(ed)]
μ_marg = marginalize(μ,1)
marg = marginals(μ_marg)[time]

ed_fourier = inedges(bp_fourier.g, node) |> first
@show ed_fourier
μ_fourier = bp_fourier.μ[idx(ed_fourier)]
μ_fourier_marg = marginalize(μ_fourier,1)
marg_fourier = marginals(μ_fourier_marg)[time]

@show marg
@show marg_fourier;

ed = Indexed Edge 1 => 2 with index 1
ed_fourier = Indexed Edge 1 => 2 with index 1
marg = [0.10761390792083407, 0.892386092079166]
marg_fourier = [0.10864933087371964, 0.8913506691262804]


In [181]:
node = 1
time = 3

edd = outedges(bp.g, node)
for ed in edd
    display(ed)
    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    marg = marginals(μ_marg)[time]

    μ_fourier = bp_fourier.μ[idx(ed)]
    μ_fourier_marg = marginalize(μ_fourier,1)
    marg_fourier = marginals(μ_fourier_marg)[time]

    display(marg)
    display(marg_fourier)
end

Indexed Edge 1 => 2 with index 1

2-element Vector{Float64}:
 0.10761390792083407
 0.892386092079166

2-element Vector{Float64}:
 0.10864933087371964
 0.8913506691262804

Indexed Edge 1 => 3 with index 2

2-element Vector{Float64}:
 0.10761390792083407
 0.892386092079166

...

2-element Vector{Float64}:
 0.10761390792083177
 0.8923860920791682

2-element Vector{Float64}:
 0.10864969559657056
 0.8913503044034294

Indexed Edge 1 => 5 with index 4

2-element Vector{Float64}:
 0.10761390792083152
 0.8923860920791684

2-element Vector{Float64}:
 0.1086514325412967
 0.8913485674587033

In [182]:
for ed in edges(bp.g)
    display(ed)

    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    μ_fourier = bp_fourier.μ[idx(ed)]
    μ_fourier_marg = marginalize(μ_fourier,1)

    for time in 1:T+1
        marg = marginals(μ_marg)[time]
        marg_fourier = marginals(μ_fourier_marg)[time]
        display("time = $time")
        display(marg)
        display(marg_fourier)
    end
end

Indexed Edge 1 => 2 with index 1

"time = 1"

2-element Vector{Float64}:
 0.0
 1.0

2-element Vector{Float64}:
 1.2221863724321797e-19
 1.0

"time = 2"

...

2-element Vector{Float64}:
 0.5
 0.5

2-element Vector{Float64}:
 0.5000000000000001
 0.49999999999999994

"time = 5"

2-element Vector{Float64}:
 0.5000000000000001
 0.4999999999999999

2-element Vector{Float64}:
 0.5
 0.5000000000000001